<a href="https://colab.research.google.com/github/DomMcOyle/TACO-expl/blob/add_detr/Training%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4CV project work

Summary: Improving and explaining instance segmentation on a litter detection dataset

Members:
- Dell'Olio Domenico
- Delvecchio Giovanni Pio
- Disabato Raffaele

The project was developed in order to improve instance segmentation results on the [TACO Dataset](http://tacodataset.org/).

We decided to implement and test various architectures, among the highest scoring on COCO instance segmentation datasets, in order to compare their performances.
We also tested some explainability methods on these models to try and explain model predictions.

## This notebook contains:
- Dataset splitting and label replacing

In [1]:
# repository cloning
!git clone https://github.com/DomMcOyle/TACO-expl
%cd /content/TACO-expl/
!git checkout add_detr
%cd /content/

Cloning into 'TACO-expl'...
remote: Enumerating objects: 2490, done.
remote: Counting objects: 100% (652/652), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 2490 (delta 524), reused 581 (delta 476), pack-reused 1838
Receiving objects: 100% (2490/2490), 186.80 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (1292/1292), done.
/content/TACO-expl
Branch 'add_detr' set up to track remote branch 'add_detr' from 'origin'.
Switched to a new branch 'add_detr'
/content


In [7]:
# library imports
%cd TACO-expl

import torch
import json
import copy
from sklearn.model_selection import train_test_split

[Errno 2] No such file or directory: 'TACO-expl'
/content/TACO-expl


## Dataset splitting and label replacing
As first pre-processing step, we obtain the so-called TACO-10 dataset from the official one provided. This dataset, differently from the ready-to-use one, has only 10 segmentation classes obtained by unifying all the available fine-grained classes.

The ten new target classes are: "Bottle", "Bottle cap", "Can", "Cigarette", "Cup", "Lid", "Other", "Plastic bag & wrapper", "Pop tab", "Straw".

The following code was adapted from the [TACO github repository](https://github.com/pedropro/TACO).

In [4]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  """
  Function creating a map for the classes substitution
  :param original: original categories of the dataset as described by the value "categories" in the dataset json
  :param keep_supercategories: list of super-categories to keep in the final dataset
  :return : a dictionary in the form "category_to_replace" : "category_used_as_replacement"
  """
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """
      Replaces classes of dataset based on a dictionary

      :param dataset: dataset description as obtained from the annotation json
      :param class_map: mapping determining which class has to be substituted. Output of create_map.
      """
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]


In [5]:
with open("/content/TACO-expl/data/annotations.json", "r") as f:
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
print(dataset["categories"])

[{'supercategory': '', 'id': 1, 'name': 'Bottle'}, {'supercategory': '', 'id': 2, 'name': 'Bottle cap'}, {'supercategory': '', 'id': 3, 'name': 'Can'}, {'supercategory': '', 'id': 4, 'name': 'Cigarette'}, {'supercategory': '', 'id': 5, 'name': 'Cup'}, {'supercategory': '', 'id': 6, 'name': 'Lid'}, {'supercategory': '', 'id': 7, 'name': 'Other'}, {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'}, {'supercategory': '', 'id': 9, 'name': 'Pop tab'}, {'supercategory': '', 'id': 10, 'name': 'Straw'}]


After mapping the classes, a sanity check is employed on the dataset and it is then split in 80/10/10 proportion in Training, Validation and Test splits.

The sanity check consists in removing all the images without boxes or with only ill-formed ones. If the image has some usable boxed and some unusable ones, the latter will be removed by the dataset loading class.

It's important to notice that we employed only the "official" version of the dataset, which comprises 1500 images. We also tried using the "unofficial" one, which contains almoost 4000 samples, but since most of them were uploaded by unexpert users, the quality of segmentations and detections was extremely poor.




In [9]:
args = {
    "nr_trials":1, # change if you want to generate more than one split
    "test_percentage":0.1,
    "split_seed": 42,
    "val_percentage":0.1,
    "dataset_dir":'/content/TACO-expl/data'
}

# annotation path
ann_input_path = args["dataset_dir"] + '/annotations.json'

# Load annotations
with open(ann_input_path, 'r') as f:
    dataset = json.loads(f.read())

# execute class mapping
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

if keep_categories is not None:
  class_map = create_map(dataset["categories"], keep_categories)
  replace_dataset_classes(dataset, class_map)

anns = dataset['annotations']
scene_anns = dataset['scene_annotations']
imgs = dataset['images']
nr_images = len(imgs)

dimensions = {im['id']: [im['height'], im['width']] for im in dataset['images']}
bad_ann = []
image_with_at_least_one_ann = set()
# check sanity of annotation
for a in anns:
    h, w = dimensions[a["image_id"]]
    boxes = a["bbox"]
    # guard against no boxes via resizing
    boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
    boxes[:, 2:] += boxes[:, :2]
    boxes[:, 0::2].clamp_(min=0, max=w)
    boxes[:, 1::2].clamp_(min=0, max=h)

    segmentations = True if a["segmentation"] else False

    keep = (boxes[:, 3] > boxes[:, 1]) & (boxes[:, 2] > boxes[:, 0])
    if (keep and segmentations):
      image_with_at_least_one_ann.add(a["image_id"])
    else:
      bad_ann.append(a)


imgs = []
# if image is without boxes is removed
for i in dataset['images']:
  if i["id"] in image_with_at_least_one_ann:
    imgs.append(i)
  else:
    print("removed image:")
    print(i)
    print()
print(bad_ann)

for i in range(args["nr_trials"]):

    # create new dataset placeholder
    train_set = {
        'info': None,
        'images': [],
        'annotations': [],
        'scene_annotations': [],
        'licenses': [],
        'categories': [],
        'scene_categories': [],
    }
    train_set['info'] =  dataset['info']
    train_set['categories'] = dataset['categories']
    train_set['scene_categories'] = dataset['scene_categories']

    val_set = copy.deepcopy(train_set)
    test_set = copy.deepcopy(train_set)

    train_set['images'], partial = train_test_split(imgs,
                                                    random_state=args["split_seed"],
                                                               test_size=args["test_percentage"]+args["val_percentage"])
    val_set['images'], test_set["images"] = train_test_split(partial,
                                                             random_state=args["split_seed"],
                                                             test_size=args["test_percentage"]/(args["test_percentage"]+args["val_percentage"]))

    # Aux Image Ids to split annotations
    test_img_ids, val_img_ids, train_img_ids = [],[],[]
    for img in test_set['images']:
        test_img_ids.append(img['id'])

    for img in val_set['images']:
        val_img_ids.append(img['id'])

    for img in train_set['images']:
        train_img_ids.append(img['id'])

    # Split instance annotations
    for ann in anns:
        if ann['image_id'] in test_img_ids:
            test_set['annotations'].append(ann)
        elif ann['image_id'] in val_img_ids:
            val_set['annotations'].append(ann)
        elif ann['image_id'] in train_img_ids:
            train_set['annotations'].append(ann)

    # Split scene tags
    for ann in scene_anns:
        if ann['image_id'] in test_img_ids:
            test_set['scene_annotations'].append(ann)
        elif ann['image_id'] in val_img_ids:
            val_set['scene_annotations'].append(ann)
        elif ann['image_id'] in train_img_ids:
            train_set['scene_annotations'].append(ann)

    # Write dataset splits
    ann_train_out_path = args["dataset_dir"] + '/' + 'annotations_off_' + str(i) +'_train.json'
    ann_val_out_path   = args["dataset_dir"] + '/' + 'annotations_off_' + str(i) + '_val.json'
    ann_test_out_path  = args["dataset_dir"] + '/' + 'annotations_off_' + str(i) + '_test.json'

    with open(ann_train_out_path, 'w+') as f:
        f.write(json.dumps(train_set))

    with open(ann_val_out_path, 'w+') as f:
        f.write(json.dumps(val_set))

    with open(ann_test_out_path, 'w+') as f:
        f.write(json.dumps(test_set))


[]
